### resize img and annotation

In [8]:
import os
import json
from PIL import Image

def resize_and_normalize_data(new_height, new_width, old_height= None, old_width= None, rawdata_path='rawdata', normalized_data_path='normalized_data'):
    """
    Resizes images and updates bounding box information in JSON files for all subfolders within the rawdata directory.
    Saves the processed images and JSON files into normalized_data, preserving the folder structure.
    
    Parameters:
    - new_height (int): The desired height for the resized images.
    - new_width (int): The desired width for the resized images.
    - rawdata_path (str): The path to the 'rawdata' directory. Defaults to 'rawdata'.
    - normalized_data_path (str): The path to the 'normalized_data' directory where processed files will be saved. Defaults to 'normalized_data'.
    """
    
    if not os.path.isdir(rawdata_path):
        print(f"Error: Rawdata folder '{rawdata_path}' does not exist.")
        return
    
    # Iterate over all subdirectories in rawdata_path
    for subdir, dirs, files in os.walk(rawdata_path):
        # Determine the relative path to maintain folder structure
        rel_path = os.path.relpath(subdir, rawdata_path)
        
        # Define the corresponding path in normalized_data
        target_subdir = os.path.join(normalized_data_path, rel_path)
        
        # Create the target subdirectory if it doesn't exist
        os.makedirs(target_subdir, exist_ok=True)
        
        print(f"\nProcessing folder: {subdir} -> {target_subdir}")
        
        for file in files:
            file_path = os.path.join(subdir, file)
            target_file_path = os.path.join(target_subdir, file)
            filename, ext = os.path.splitext(file)
            
            # Process image files
            if ext.lower() in ['.jpg', '.jpeg', '.png']:
                try:
                    with Image.open(file_path) as img:
                        original_width, original_height = img.size
                        resized_img = img.resize((new_width, new_height), Image.ANTIALIAS)
                        resized_img.save(target_file_path)
                        print(f"  Resized image: {file} from ({original_width}, {original_height}) to ({new_width}, {new_height})")
                except Exception as e:
                    print(f"  Error processing image {file}: {e}")
            
            # Process JSON files
            elif ext.lower() == '.json':
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                    
                    # Extract original dimensions
                    if 'imageHeight' in data:
                        orig_height = data['imageHeight']
                    else:
                        orig_height = old_height

                    
                    if 'imageWidth' in data:
                        orig_width = data.get('imageWidth')
                    else:
                        orig_width = old_width
                    
                    if orig_height is None or orig_width is None:
                        print(f"  Original dimensions not found in {file}. Skipping JSON update.")
                        continue
                    
                    # Calculate scaling factors
                    height_scale = new_height / orig_height
                    width_scale = new_width / orig_width
                    
                    # Update image dimensions
                    data['imageHeight'] = new_height
                    data['imageWidth'] = new_width
                    
                    # Update each shape's points
                    for shape in data.get('shapes', []):
                        new_points = []
                        for point in shape.get('points', []):
                            if len(point) != 2:
                                print(f"    Invalid point format in {file}: {point}. Skipping this point.")
                                continue
                            x, y = point
                            new_x = x * width_scale
                            new_y = y * height_scale
                            new_points.append([new_x, new_y])
                        shape['points'] = new_points
                        
                    # Optionally, update the imagePath if necessary
                    # For example, if imagePath is relative and images are now in normalized_data
                    # data['imagePath'] = os.path.relpath(target_image_path, target_subdir)
                    
                    # Save the updated JSON
                    with open(target_file_path, 'w', encoding='utf-8') as f:
                        json.dump(data, f, indent=2)
                    
                    print(f"  Updated JSON: {file}")
                
                except Exception as e:
                    print(f"  Error processing JSON {file}: {e}")
    
    print("\nAll folders have been processed and saved to normalized_data.")

# Example usage:
resize_and_normalize_data(1440, 1080, old_height= 792, old_width= 612, rawdata_path= '10Trail_Revised')



Processing folder: 10Trail_Revised -> normalized_data/.

Processing folder: 10Trail_Revised/adfm.201702654 -> normalized_data/adfm.201702654
  Updated JSON: adfm.2017026540003.json
  Updated JSON: adfm.2017026540002.json


/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/1833642774.py:44: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_img = img.resize((new_width, new_height), Image.ANTIALIAS)


  Resized image: adfm.2017026540008.jpg from (4134, 5512) to (1080, 1440)
  Resized image: adfm.2017026540009.jpg from (4134, 5512) to (1080, 1440)
  Updated JSON: adfm.2017026540009.json
  Updated JSON: adfm.2017026540005.json
  Updated JSON: adfm.2017026540012.json
  Updated JSON: adfm.2017026540004.json
  Updated JSON: adfm.2017026540008.json
  Updated JSON: adfm.2017026540011.json
  Updated JSON: adfm.2017026540007.json
  Updated JSON: adfm.2017026540006.json
  Updated JSON: adfm.2017026540010.json
  Resized image: adfm.2017026540010.jpg from (4134, 5512) to (1080, 1440)
  Resized image: adfm.2017026540004.jpg from (4134, 5512) to (1080, 1440)
  Updated JSON: adfm.2017026540001.json
  Resized image: adfm.2017026540005.jpg from (4134, 5512) to (1080, 1440)
  Resized image: adfm.2017026540011.jpg from (4134, 5512) to (1080, 1440)
  Resized image: adfm.2017026540007.jpg from (4134, 5512) to (1080, 1440)
  Resized image: adfm.2017026540012.jpg from (4134, 5512) to (1080, 1440)
  Resize

### Process the json files for model expected assistant output

In [9]:
import os
import json
import random
def process_json_file(input_json_path, output_txt_path, shuffle_data=False, seed=1314):
    with open(input_json_path, 'r') as f:
        data = json.load(f)
    
    transformed_data = []

    shapes = data.get("shapes", [])
    for idx, shape in enumerate(shapes, start=1):
        label = shape.get("label", "")
        points = shape.get("points", [])

        if len(points) != 2:
            print(f"Warning: Shape {shape.get('shape_id', '')} in {input_json_path} does not have exactly 2 points.")
            continue

        # Extract coordinates
        x1, y1 = points[0]
        x2, y2 = points[1]
        
        # Create tuple (label, x1, y1, x2, y2, idx)
        obj = (label, round(x1), round(y1), round(x2), round(y2), idx)
        transformed_data.append(obj)
    
    # Ensure the output directory exists
    if shuffle_data:
        if seed is not None:
            random.seed(seed)  # Set seed for reproducibility
        random.shuffle(transformed_data)
    os.makedirs(os.path.dirname(output_txt_path), exist_ok=True)
    
    # Write the entire list of tuples as a single string
    with open(output_txt_path, 'w') as f:
        f.write(str(transformed_data))

def process_all_jsons(input_parent_folder, output_parent_folder):
    # Iterate through each subfolder in the input parent folder
    for subdir, _, files in os.walk(input_parent_folder):
        # Determine the relative path to maintain folder structure
        rel_path = os.path.relpath(subdir, input_parent_folder)
        # Define the corresponding output subdirectory
        output_subdir = os.path.join(output_parent_folder, rel_path)
        
        for file in files:
            if file.lower().endswith('.json'):
                input_json_path = os.path.join(subdir, file)
                
                # Define the output TXT file name, e.g., originalname_transformed.txt
                filename, _ = os.path.splitext(file)
                output_filename = f"{filename}_transformed.txt"
                output_txt_path = os.path.join(output_subdir, output_filename)
                
                print(f"Processing {input_json_path} -> {output_txt_path}")
                process_json_file(input_json_path, output_txt_path)

# Define input and output parent folders
input_parent_folder = "normalized_data"               # Replace with the correct input path if necessary
output_parent_folder = "processed_json_assistant"         # Output folder for transformed TXT files

# Create the output parent folder if it doesn't exist
os.makedirs(output_parent_folder, exist_ok=True)

# Process all JSON files
process_all_jsons(input_parent_folder, output_parent_folder)


Processing normalized_data/adfm.201702654/adfm.2017026540003.json -> processed_json_assistant/adfm.201702654/adfm.2017026540003_transformed.txt
Processing normalized_data/adfm.201702654/adfm.2017026540002.json -> processed_json_assistant/adfm.201702654/adfm.2017026540002_transformed.txt
Processing normalized_data/adfm.201702654/adfm.2017026540009.json -> processed_json_assistant/adfm.201702654/adfm.2017026540009_transformed.txt
Processing normalized_data/adfm.201702654/adfm.2017026540005.json -> processed_json_assistant/adfm.201702654/adfm.2017026540005_transformed.txt
Processing normalized_data/adfm.201702654/adfm.2017026540012.json -> processed_json_assistant/adfm.201702654/adfm.2017026540012_transformed.txt
Processing normalized_data/adfm.201702654/adfm.2017026540004.json -> processed_json_assistant/adfm.201702654/adfm.2017026540004_transformed.txt
Processing normalized_data/adfm.201702654/adfm.2017026540008.json -> processed_json_assistant/adfm.201702654/adfm.2017026540008_transfor

### A function to draw bounding boxes given json file and img path (Not necessary for data processing)

In [26]:
import json
from PIL import Image, ImageDraw, ImageFont
import os
import hashlib

def generate_color(label):
    """
    Generates a unique color for a given label using a hash function.
    
    Args:
        label (str): The label for which to generate a color.
        
    Returns:
        tuple: A tuple representing an RGB color.
    """
    # Use SHA-256 hash to generate a consistent hash for the label
    hash_object = hashlib.sha256(label.encode())
    hex_dig = hash_object.hexdigest()
    
    # Take the first 6 characters for RGB
    r = int(hex_dig[0:2], 16)
    g = int(hex_dig[2:4], 16)
    b = int(hex_dig[4:6], 16)
    
    return (r, g, b)

def draw_bounding_boxes(json_path, image_path, output_path=None):
    """
    Draws bounding boxes on an image based on the provided JSON file.

    Args:
        json_path (str): Path to the JSON file containing bounding box data.
        image_path (str): Path to the input image.
        output_path (str, optional): Path to save the output image. 
                                     If not provided, appends '_boxed' to the original image name.
    """
    # Load JSON data
    try:
        with open(json_path, 'r') as f:
            data = json.load(f)
    except Exception as e:
        print(f"Error reading JSON file: {e}")
        return

    # Open the image
    try:
        image = Image.open(image_path).convert("RGB")
    except Exception as e:
        print(f"Error opening image file: {e}")
        return

    draw = ImageDraw.Draw(image)

    # Predefined colors for known labels
    predefined_label_colors = {
        "HeaderMatter": (255, 0, 0),      # Red
        "FooterMatter": (0, 0, 255),      # Blue
        "Heading": (0, 128, 0),           # Green
        "text": (255, 255, 0)             # Yellow
    }

    # Dictionary to store colors for labels
    label_colors = predefined_label_colors.copy()

    # Function to get color for a label
    def get_color(label):
        if label in label_colors:
            return label_colors[label]
        else:
            # Assign a new color
            color = generate_color(label)
            label_colors[label] = color
            return color

    # Load a font
    try:
        # You might need to specify the full path to a .ttf font file on your system
        font = ImageFont.truetype("arial.ttf", size=15)
    except IOError:
        # If the specified font is not found, load the default font
        font = ImageFont.load_default()

    # Iterate through each object and draw bounding boxes
    for obj in data.get("objects", []):
        bbox = obj.get("boundingBox", {})
        label = obj.get("label", "N/A")

        x1 = bbox.get("x1", 0)
        y1 = bbox.get("y1", 0)
        x2 = bbox.get("x2", 0)
        y2 = bbox.get("y2", 0)

        # Get color for the label
        color = get_color(label)

        # Draw the rectangle
        draw.rectangle([(x1, y1), (x2, y2)], outline=color, width=2)

        # Optionally, draw the label
        text = label
        text_size = draw.textsize(text, font=font)
        text_background = [x1, y1 - text_size[1], x1 + text_size[0], y1]
        draw.rectangle(text_background, fill=color)
        draw.text((x1, y1 - text_size[1]), text, fill="black", font=font)

    # Determine output path
    if not output_path:
        base, ext = os.path.splitext(image_path)
        output_path = f"{base}_boxed{ext}"

    # Save the image
    try:
        image.save(output_path)
        print(f"Image saved to {output_path}")
    except Exception as e:
        print(f"Error saving image: {e}")

# Example Usage

json_file = "/Users/zl/Desktop/courses/cis 8000/qwen2vl_data_processingv2/milestone4_plot.json"      # Path to your JSON file
input_image = "/Users/zl/Desktop/courses/cis 8000/qwen2vl_data_processingv2/test_data/images/adfm.201706950/adfm.2017069500000.jpg"        # Path to your input image
output_image = "output_image.jpg"      # (Optional) Path for the output image

draw_bounding_boxes(json_file, input_image, output_image)


Image saved to output_image.jpg


/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_33588/1757315223.py:101: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(text, font=font)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_33588/1757315223.py:101: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(text, font=font)


In [ ]:
Example below:
{{
    "objects": [
        {{
            "readingOrder": 1,
            "label": "",
            "boundingBox": {{
                "x1": ,
                "y1": ,
                "x2": ,
                "y2": 
            }}
        }},
        {{
            "readingOrder": 2,
            "label": "",
            "boundingBox": {{
                "x1": ,
                "y1": ,
                "x2": ,
                "y2": 
            }}
        }}
    ]
}}

In [7]:
import numpy as np
np.unique(all_lab)

array(['Figure', 'FigureCaption', 'FooterMatter', 'HeaderMatter',
       'Heading', 'MathBlock', 'PageNumber', 'Table', 'TableCaption',
       'Title', 'reference', 'text'], dtype='<U13')

### Create user input


In [25]:
import os
import json

def create_text_file(output_txt_path, image_width, image_height):
    """
    Creates a text file with image width and height.
    """
    with open(output_txt_path, 'w') as f:
        text = f"""<OD><image>"""
        f.write(text)


def process_json_to_text(input_json_path, output_txt_path):
    """
    Reads the JSON file, extracts imageWidth and imageHeight,
    and writes them to a text file.
    """
    try:
        with open(input_json_path, 'r') as f:
            data = json.load(f)
        
        image_width = data.get("imageWidth", "Unknown")
        image_height = data.get("imageHeight", "Unknown")
        
        create_text_file(output_txt_path, image_width, image_height)
        print(f"Created text file: {output_txt_path}")
        
    except Exception as e:
        print(f"Error processing {input_json_path}: {e}")

def process_all_jsons(input_parent_folder, output_parent_folder):
    """
    Iterates through all JSON files in the input_parent_folder and
    creates corresponding text files in the output_parent_folder.
    """
    for subdir, _, files in os.walk(input_parent_folder):
        
        for file in files:

            if file.lower().endswith('.json') or file.lower().endswith('.txt'):
                input_json_path = os.path.join(subdir, file)
                
                # Determine the relative path to maintain folder structure
                rel_path = os.path.relpath(subdir, input_parent_folder)
                output_subdir = os.path.join(output_parent_folder, rel_path)
                
                # Ensure the output subdirectory exists
                os.makedirs(output_subdir, exist_ok=True)
                
                # Define the output text file name, e.g., originalname.txt
                filename, _ = os.path.splitext(file)
                
                output_filename = f"{filename}.txt"
                output_txt_path = os.path.join(output_subdir, output_filename)

                create_text_file(output_txt_path, None, None)
                # Process the JSON and create the text file
                # process_json_to_text(input_json_path, output_txt_path)


input_parent_folder = "processed_json_assistant"       # Source folder with transformed JSON files
output_parent_folder = "processed_json_user" # Destination folder for text files

# Create the output parent folder if it doesn't exist
os.makedirs(output_parent_folder, exist_ok=True)

process_all_jsons(input_parent_folder, output_parent_folder)
print("All text files have been created in 'processed_json_user' folder.")


All text files have been created in 'processed_json_user' folder.


### Move the images into train_data folder (Deprecated. Use the split_dataset function done below for train_test split into two folders)

### Split normalized_data folder images into train_data and test_data. Only for images not json

In [5]:
import os
import shutil
import random

def split_dataset(input_parent_folder, train_output_folder, test_output_folder, train_percentage=80, seed=42):
    """
    Splits images from the input_parent_folder into training and testing sets.
    The split is based on the train_percentage provided.

    Args:
        input_parent_folder (str): Source folder containing images.
        train_output_folder (str): Destination folder for training images.
        test_output_folder (str): Destination folder for testing images.
        train_percentage (float): Percentage of images to be used for training (0-100).
        seed (int): Seed for random number generator to ensure reproducibility.
    """
    # Supported image extensions
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff'}

    # Ensure train and test output folders exist
    os.makedirs(train_output_folder, exist_ok=True)
    os.makedirs(test_output_folder, exist_ok=True)

    # Set random seed for reproducibility
    random.seed(seed)
    total_train = 0
    total_test = 0
    for subdir, _, files in os.walk(input_parent_folder):
        # Filter image files
        image_files = [f for f in files if os.path.splitext(f.lower())[1] in image_extensions]
        
        if not image_files:
            continue  # Skip subdirectories without images

        # Shuffle the list of images
        random.shuffle(image_files)

        # Determine split index
        split_idx = int(len(image_files) * (train_percentage / 100))

        # Split into train and test
        train_files = image_files[:split_idx]
        test_files = image_files[split_idx:]
        total_train += len(train_files)
        total_test += len(test_files)
        # Calculate relative path to maintain folder structure
        rel_path = os.path.relpath(subdir, input_parent_folder)

        # Define corresponding subdirectories in train and test folders
        train_subdir = os.path.join(train_output_folder, rel_path)
        test_subdir = os.path.join(test_output_folder, rel_path)

        # Create subdirectories if they don't exist
        os.makedirs(train_subdir, exist_ok=True)
        os.makedirs(test_subdir, exist_ok=True)

        # Copy training images
        for file in train_files:
            src_path = os.path.join(subdir, file)
            dest_path = os.path.join(train_subdir, file)
            shutil.copy2(src_path, dest_path)
            print(f"Copied {src_path} to {dest_path}")

        # Copy testing images
        for file in test_files:
            src_path = os.path.join(subdir, file)
            dest_path = os.path.join(test_subdir, file)
            shutil.copy2(src_path, dest_path)
            print(f"Copied {src_path} to {dest_path}")

    print(f"Dataset split completed with {train_percentage}% training {(total_train)} imgs and {100 - train_percentage}% testing {(total_test)} imgs.")

# Usage Example
if __name__ == "__main__":
    input_parent_folder = "normalized_data"  # Source folder containing images

    # Define output folders for training and testing data
    train_output_folder = "train_data/images"
    test_output_folder = "test_data/images"

    # Create the output folders if they don't exist
    os.makedirs(train_output_folder, exist_ok=True)
    os.makedirs(test_output_folder, exist_ok=True)

    # Define the training percentage (e.g., 80% for training and 20% for testing)
    train_percentage = 80  # You can adjust this value as needed

    # Split the dataset
    split_dataset(input_parent_folder, train_output_folder, test_output_folder, train_percentage)

    print("All images have been split into 'train_data/images' and 'test_data/images' folders.")


Copied normalized_data/adfm.201702654/adfm.2017026540012.jpg to train_data/images/adfm.201702654/adfm.2017026540012.jpg
Copied normalized_data/adfm.201702654/adfm.2017026540007.jpg to train_data/images/adfm.201702654/adfm.2017026540007.jpg
Copied normalized_data/adfm.201702654/adfm.2017026540010.jpg to train_data/images/adfm.201702654/adfm.2017026540010.jpg
Copied normalized_data/adfm.201702654/adfm.2017026540002.jpg to train_data/images/adfm.201702654/adfm.2017026540002.jpg
Copied normalized_data/adfm.201702654/adfm.2017026540000.jpg to train_data/images/adfm.201702654/adfm.2017026540000.jpg
Copied normalized_data/adfm.201702654/adfm.2017026540011.jpg to train_data/images/adfm.201702654/adfm.2017026540011.jpg
Copied normalized_data/adfm.201702654/adfm.2017026540001.jpg to train_data/images/adfm.201702654/adfm.2017026540001.jpg
Copied normalized_data/adfm.201702654/adfm.2017026540006.jpg to train_data/images/adfm.201702654/adfm.2017026540006.jpg
Copied normalized_data/adfm.201702654/ad

### Putting system, assistant, user all together (for finetuning)

In [26]:
import os
import json

def read_system_instruction(system_instruction_path):
    """
    Reads the system instruction from the specified text file.
    """
    try:
        with open(system_instruction_path, 'r', encoding='utf-8') as f:
            system_content = f.read().strip()
            if not system_content:
                raise ValueError("System instruction file is empty.")
            return system_content
    except Exception as e:
        print(f"Error reading system instruction from {system_instruction_path}: {e}")
        return "You are a helpful assistant."  # Fallback system instruction

def read_user_text(txt_file_path):
    """
    Reads the user text from the corresponding txt file.
    """
    try:
        with open(txt_file_path, 'r', encoding='utf-8') as f:
            user_content = f.read().strip()
            if not user_content:
                raise ValueError("User instruction file is empty.")
            return user_content
    except Exception as e:
        print(f"Error reading {txt_file_path}: {e}")
        return ""

def read_assistant_text(txt_file_path):
    """
    Reads the read_assistant text from the corresponding txt file.
    """
    try:
        with open(txt_file_path, 'r', encoding='utf-8') as f:
            ass_content = f.read().strip()
            if not ass_content:
                raise ValueError("ass_content content file is empty.")
            return ass_content
    except Exception as e:
        print(f"Error reading {txt_file_path}: {e}")
        return ""
        

def create_dataset(train_data_folder, user_text_folder, assistant_text_folder, system_instruction_path, output_json_path):
    """
    Creates the JSON dataset as per the specified format, incorporating system instructions.
    """
    dataset = []
    images_folder = os.path.join(train_data_folder, "images")
    
    # Read system instruction
    system_content = read_system_instruction(system_instruction_path)
    
    for subdir, _, files in os.walk(images_folder):
        for file in files:
            if os.path.splitext(file.lower())[1] in ['.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff']:
                image_path = os.path.join(subdir, file).replace("\\", "/")  # For Windows compatibility
                rel_image_path = os.path.relpath(image_path, train_data_folder).replace("\\", "/")
                
                base_name, _ = os.path.splitext(file)
                
                # Corresponding txt file
                user_txt_file = os.path.join(user_text_folder, os.path.relpath(subdir, images_folder), f"{base_name}_transformed.txt")
                
                # Corresponding assistant text file
                assistant_text_file = os.path.join(assistant_text_folder, os.path.relpath(subdir, images_folder), f"{base_name}_transformed.txt")
                
                # Read user text
                user_text = read_user_text(user_txt_file)
                
                # Generate assistant text
                assistant_text = read_assistant_text(assistant_text_file)
                # Construct the JSON entry
                entry = {
                    "conversations": [
                        {
                            "from": "system",
                            "value": system_content
                        },
                        {
                            "from": "human",
                            "value": user_text
                        },
                        {
                            "from": "gpt",
                            "value": assistant_text
                        }
                    ],
                    "images": [
                        os.path.join("images", os.path.relpath(image_path, images_folder)).replace("\\", "/")
                    ]
                }
                
                dataset.append(entry)
                print(f"Added entry for {image_path}")
    
    # Write the dataset to a JSON file
    try:
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump(dataset, f, ensure_ascii=False, indent=4)
        print(f"Dataset has been created at {output_json_path}")
    except Exception as e:
        print(f"Error writing dataset to {output_json_path}: {e}")

if __name__ == "__main__":
    train_data_folder = "train_data"                     # Folder containing 'images' Change to train_data or test_data
    user_text_folder = "processed_json_user"             # Folder containing user .txt files
    assistant_json_folder = "processed_json_assistant"              # Folder containing assistant JSON files
    system_instruction_path = "system_instruction.txt"    # Path to system_instruction.txt
    output_json_path = os.path.join(train_data_folder, "dataset.json")  # Output JSON file
    
    create_dataset(train_data_folder, user_text_folder, assistant_json_folder, system_instruction_path, output_json_path)


Added entry for train_data/images/adfm.201702654/adfm.2017026540010.jpg
Added entry for train_data/images/adfm.201702654/adfm.2017026540004.jpg
Added entry for train_data/images/adfm.201702654/adfm.2017026540005.jpg
Added entry for train_data/images/adfm.201702654/adfm.2017026540011.jpg
Added entry for train_data/images/adfm.201702654/adfm.2017026540007.jpg
Added entry for train_data/images/adfm.201702654/adfm.2017026540012.jpg
Added entry for train_data/images/adfm.201702654/adfm.2017026540006.jpg
Added entry for train_data/images/adfm.201702654/adfm.2017026540002.jpg
Added entry for train_data/images/adfm.201702654/adfm.2017026540001.jpg
Added entry for train_data/images/adfm.201702654/adfm.2017026540000.jpg
Added entry for train_data/images/adfm.201600681/adfm.2016006810000.jpg
Added entry for train_data/images/adfm.201600681/adfm.2016006810001.jpg
Added entry for train_data/images/adfm.201600681/adfm.2016006810005.jpg
Added entry for train_data/images/adfm.201600681/adfm.2016006810

### Transform all pdf paper from papers folder to jpg images with certain height and width. (Not necessary for data processing)



In [6]:
import os
from pdf2image import convert_from_path
from PIL import Image
import argparse

def convert_pdfs_to_images(papers_dir, output_dir, new_width, new_height, poppler_path=None):
    """
    Converts all PDFs in the papers_dir to images, resizes them, and saves them in output_dir.
    
    Args:
        papers_dir (str): Path to the directory containing PDF files.
        output_dir (str): Path to the directory where images will be saved.
        new_width (int): The desired width of the output images.
        new_height (int): The desired height of the output images.
        poppler_path (str, optional): Path to the Poppler bin directory. Required on Windows.
    """
    if not os.path.exists(papers_dir):
        print(f"The directory {papers_dir} does not exist.")
        return

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created output directory at {output_dir}")

    # Iterate over all PDF files in the papers directory
    for filename in os.listdir(papers_dir):
        if filename.lower().endswith('.pdf'):
            pdf_path = os.path.join(papers_dir, filename)
            pdf_name = os.path.splitext(filename)[0]  # Remove .pdf extension
            ### CHANGED output_subdir = os.path.join(output_dir, pdf_name)
            output_subdir = output_dir
            # Create subdirectory for the current PDF
            if not os.path.exists(output_subdir):
                os.makedirs(output_subdir)
                print(f"Created directory for {pdf_name} at {output_subdir}")

            try:
                # Convert PDF to images
                print(f"Converting {pdf_path} to images...")
                images = convert_from_path(pdf_path, poppler_path=poppler_path)
                
                for i, image in enumerate(images, start=1):
                    # Resize image
                    resized_image = image.resize((new_width, new_height), Image.ANTIALIAS)
                    
                    # Create image filename with zero-padded page number
                    # CHANGED page_number = str(i - 1).zfill(4)  # Starting from 0000
                    # CHANGED image_filename = f"{pdf_name}{page_number}.jpg"
                    image_filename = f"pdfbbox_output{''.join([char for char in pdf_name if char.isdigit()])}.jpg"
                    image_path = os.path.join(output_subdir, image_filename)
                    
                    # Save image
                    resized_image.save(image_path, 'JPEG')
                    print(f"Saved image: {image_path}")
            
            except Exception as e:
                print(f"Failed to convert {pdf_path}: {e}")

if __name__ == "__main__":
    
    convert_pdfs_to_images(
        # papers_dir='../papers',
        papers_dir='./10Trail_Revised/pdfs',
        # output_dir='all_transformed',
        output_dir='./10Trail_Revised/pdfs',
        new_width=1080,
        new_height=1440,
        poppler_path=None
    )


Converting ./10Trail_Revised/pdfs/test_16.pdf to images...


/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/1013859811.py:44: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_image = image.resize((new_width, new_height), Image.ANTIALIAS)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/1013859811.py:44: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_image = image.resize((new_width, new_height), Image.ANTIALIAS)


Saved image: ./10Trail_Revised/pdfs/pdfbbox_output16.jpg
Converting ./10Trail_Revised/pdfs/test_8.pdf to images...
Saved image: ./10Trail_Revised/pdfs/pdfbbox_output8.jpg
Converting ./10Trail_Revised/pdfs/test_9.pdf to images...


/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/1013859811.py:44: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_image = image.resize((new_width, new_height), Image.ANTIALIAS)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/1013859811.py:44: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_image = image.resize((new_width, new_height), Image.ANTIALIAS)


Saved image: ./10Trail_Revised/pdfs/pdfbbox_output9.jpg
Converting ./10Trail_Revised/pdfs/test_17.pdf to images...
Saved image: ./10Trail_Revised/pdfs/pdfbbox_output17.jpg
Converting ./10Trail_Revised/pdfs/test_29.pdf to images...


...

/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/1013859811.py:44: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_image = image.resize((new_width, new_height), Image.ANTIALIAS)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/1013859811.py:44: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_image = image.resize((new_width, new_height), Image.ANTIALIAS)


Saved image: ./10Trail_Revised/pdfs/pdfbbox_output26.jpg
Converting ./10Trail_Revised/pdfs/test_32.pdf to images...
Saved image: ./10Trail_Revised/pdfs/pdfbbox_output32.jpg
Converting ./10Trail_Revised/pdfs/test_4.pdf to images...


/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/1013859811.py:44: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_image = image.resize((new_width, new_height), Image.ANTIALIAS)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/1013859811.py:44: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_image = image.resize((new_width, new_height), Image.ANTIALIAS)


Saved image: ./10Trail_Revised/pdfs/pdfbbox_output4.jpg
Converting ./10Trail_Revised/pdfs/test_5.pdf to images...
Saved image: ./10Trail_Revised/pdfs/pdfbbox_output5.jpg
Converting ./10Trail_Revised/pdfs/test_33.pdf to images...
Saved image: ./10Trail_Revised/pdfs/pdfbbox_output33.jpg
Converting ./10Trail_Revised/pdfs/test_27.pdf to images...
Saved image: ./10Trail_Revised/pdfs/pdfbbox_output27.jpg


/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/1013859811.py:44: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_image = image.resize((new_width, new_height), Image.ANTIALIAS)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/1013859811.py:44: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_image = image.resize((new_width, new_height), Image.ANTIALIAS)


### post-process the model output

In [14]:
type(eval("[('HeaderMatter', 88, 36, 995, 106, 1), ('text', 85, 134, 540, 1217, 2), ('Heading', 90, 1244, 301, 1271, 3), ('text', 91, 1279, 534, 1312, 4), ('Heading', 553, 137, 728, 159, 5), ('text', 550, 169, 996, 326, 6), ('Heading', 554, 370, 716, 394, 7), ('text', 552, 401, 814, 422, 8), ('Heading', 554, 462, 646, 493, 9), ('text', 553, 498, 999, 589, 10), ('reference', 552, 636, 999, 1323, 11), ('FooterMatter', 88, 1350, 996, 1374, 12)]")



list

In [38]:
import os
import json
import jsonlines
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont
import random

# Define paths
TEST_DATA_DIR = Path("test_data")
IMAGES_DIR = TEST_DATA_DIR
DATASET_JSON_PATH = TEST_DATA_DIR / "dataset.json"
# PREDICTIONS_JSONL_PATH = Path("generated_predictions.jsonl")  # Adjust if different
# PREDICTIONS_JSONL_PATH = Path("generated_predictions_v2_ckpt800.jsonl")
PREDICTIONS_JSONL_PATH = Path("generated_predictions_v3_ckpt400.jsonl")
# PREDICTIONS_JSONL_PATH = Path("generated_predictions_few_shot.jsonl")  # Adjust if different
# TEST_RESULT_DIR = Path("test_result_v2_ckpt800")
TEST_RESULT_DIR = Path("test_result_v3_ckpt400")

# Create the test_result directory if it doesn't exist
TEST_RESULT_DIR.mkdir(parents=True, exist_ok=True)

# Load dataset.json
with open(DATASET_JSON_PATH, 'r', encoding='utf-8') as f:
    dataset = json.load(f)

# Create a mapping from image path to ground truth
ground_truth_map = {}
unique_labels = set()  # To collect all unique labels

for entry in dataset:
    images = entry.get("images", [])
    # Assuming one image per entry; adjust if multiple
    for image_path in images:
        # Normalize image path
        image_path_normalized = image_path.replace("\\", "/")
        # Assuming the last conversation entry contains the ground truth as a list of tuples
        ground_truth = eval(entry["conversations"][-1]["value"])
        ground_truth_map[image_path_normalized] = ground_truth
        # Collect unique labels from ground truth
        for obj in ground_truth:
            if len(obj) >= 1:
                unique_labels.add(obj[0])

# Load predictions from JSONL
predictions = []
with jsonlines.open(PREDICTIONS_JSONL_PATH, 'r') as reader:
    for obj in reader:
        # Assuming 'predict' contains a string representation of a list of tuples
        predict_value = obj.get("predict", "")
        if isinstance(predict_value, str):
            try:
                prediction_tuple_list = eval(predict_value)
                predictions.append(prediction_tuple_list)
                # Collect unique labels from predictions
                for pred_obj in prediction_tuple_list:
                    if len(pred_obj) >= 1:
                        unique_labels.add(pred_obj[0])
            except Exception as e:
                print(f"Error evaluating prediction: {predict_value}")
                predictions.append([])
        elif isinstance(predict_value, list):
            # If 'predict' is already a list of tuples
            predictions.append(predict_value)
            # Collect unique labels from predictions
            for pred_obj in predict_value:
                if len(pred_obj) >= 1:
                    unique_labels.add(pred_obj[0])
        else:
            predictions.append([])
print(len(predictions))
if len(predictions) != len(ground_truth_map):
    print("Warning: Number of predictions does not match number of ground truth entries.")

# Now, iterate over images and create the test_result structure
image_paths = list(ground_truth_map.keys())
prediction_objects = predictions[:len(image_paths)]  # Truncate if necessary

# Define a list of distinguishable colors
PREDEFINED_COLORS = [
    "red", "green", "blue", "orange", "purple", "cyan",
    "magenta", "yellow", "lime", "pink", "brown", "teal",
    "olive", "maroon", "navy", "grey", "black"
]

# Initialize label to color mapping
label_color_map = {}
color_index = 0

# Function to assign a color to a label
def get_label_color(label):
    global color_index
    if label in label_color_map:
        return label_color_map[label]
    else:
        if color_index < len(PREDEFINED_COLORS):
            color = PREDEFINED_COLORS[color_index]
            color_index += 1
        else:
            # Generate a random color if predefined colors are exhausted
            color = "#{:06x}".format(random.randint(0, 0xFFFFFF))
        label_color_map[label] = color
        return color

# Load a font for labels
try:
    FONT = ImageFont.truetype("arial.ttf", size=15)
except IOError:
    FONT = ImageFont.load_default()
print('image_paths', len(image_paths))
for idx, image_rel_path in enumerate(image_paths):
    
    print(image_name)
    image_name = Path(image_rel_path).stem  # e.g., adfm.2017026540008
    image_folder = TEST_RESULT_DIR / image_name
    image_folder.mkdir(parents=True, exist_ok=True)

    # Ground truth
    ground_truth = ground_truth_map[image_rel_path]  # List of tuples
    ground_truth_path = image_folder / "ground_truth.txt"  # Changed from .json to .txt
    try:
        with open(ground_truth_path, 'w', encoding='utf-8') as gt_file:
            gt_file.write(str(ground_truth))  # Save as string representation
    except IOError as e:
        print(f"Error writing ground truth for {image_rel_path}: {e}")

    # Prediction
    if idx < len(prediction_objects):
        prediction = prediction_objects[idx]  # Already a list of tuples
    else:
        prediction = []
        print(f"No prediction found for image: {image_rel_path}")
    
    predicted_path = image_folder / "predicted.txt"  # Changed from .json to .txt
    try:
        with open(predicted_path, 'w', encoding='utf-8') as pred_file:
            pred_file.write(str(prediction))  # Save as string representation
    except IOError as e:
        print(f"Error writing prediction for {image_rel_path}: {e}")

    print(f"Processed {image_rel_path} into {image_folder}")

    # Paths to the original image
    original_image_path = IMAGES_DIR / image_rel_path
    if not original_image_path.exists():
        print(f"Original image not found: {original_image_path}")
        continue
    print('pred', prediction)
    # Open the original image
    with Image.open(original_image_path).convert("RGB") as img:
        # Create copies for ground truth and prediction
        gt_img = img.copy()
        pred_img = img.copy()

        gt_draw = ImageDraw.Draw(gt_img)
        pred_draw = ImageDraw.Draw(pred_img)

        # Draw Ground Truth Bounding Boxes
        for obj in ground_truth:
            # Each obj is a tuple: (label, x1, y1, x2, y2, reading_order)
            if len(obj) != 6:
                print(f"Invalid ground truth tuple format in {image_rel_path}: {obj}")
                continue
            label, x1, y1, x2, y2, reading_order = obj
            try:
                x1, y1, x2, y2 = float(x1), float(y1), float(x2), float(y2)
            except ValueError:
                print(f"Invalid coordinates in ground truth tuple in {image_rel_path}: {obj}")
                continue

            # Get color for the label
            color = get_label_color(label)

            # Draw rectangle
            gt_draw.rectangle([x1, y1, x2, y2], outline=color, width=2)

            # Prepare label text with reading order
            label_text = f"{label} ({reading_order})"

            # Calculate text size to create a filled rectangle behind the text for better visibility
            text_size = gt_draw.textsize(label_text, font=FONT)
            text_background = [x1, y1 - 15 if y1 - 15 > 0 else y1 + 5, x1 + text_size[0], y1]
            gt_draw.rectangle(text_background, fill=color)

            # Draw label text
            text_position = (x1, y1 - 15 if y1 - 15 > 0 else y1 + 5)
            gt_draw.text(text_position, label_text, fill="white", font=FONT)

        # Save Ground Truth Annotated Image
        gt_image_path = image_folder / "ground_truth_annotated.png"
        try:
            gt_img.save(gt_image_path, format="PNG")
        except IOError as e:
            print(f"Error saving ground truth annotated image for {image_rel_path}: {e}")

        # Draw Predicted Bounding Boxes
        for obj in prediction:
            # Each obj is a tuple: (label, x1, y1, x2, y2, reading_order)
            if len(obj) != 6:
                print(f"Invalid prediction tuple format in {image_rel_path}: {obj}")
                continue
            label, x1, y1, x2, y2, reading_order = obj
            try:
                x1, y1, x2, y2 = float(x1), float(y1), float(x2), float(y2)
            except ValueError:
                print(f"Invalid coordinates in prediction tuple in {image_rel_path}: {obj}")
                continue

            if x1 > x2 or y1 > y2:
                print(f"Invalid bounding box with x1 > x2 or y1 > y2 in prediction for {image_rel_path}: {obj}")
                continue

            # Get color for the label
            color = get_label_color(label)

            # Draw rectangle
            pred_draw.rectangle([x1, y1, x2, y2], outline=color, width=2)

            # Prepare label text with reading order
            label_text = f"{label} ({reading_order})"

            # Calculate text size to create a filled rectangle behind the text for better visibility
            text_size = pred_draw.textsize(label_text, font=FONT)
            text_background = [x1, y1, x1 + text_size[0], y1  + text_size[1]]
            pred_draw.rectangle(text_background, fill=color)

            # Draw label text
            text_position = (x1, y1)
            pred_draw.text(text_position, label_text, fill="white", font=FONT)

        # Save Predicted Annotated Image
        pred_image_path = image_folder / "predicted_annotated.png"
        try:
            pred_img.save(pred_image_path, format="PNG")
        except IOError as e:
            print(f"Error saving predicted annotated image for {image_rel_path}: {e}")

        print(f"Annotated images saved for {image_rel_path}")

print("Test results with annotated images have been organized into the 'test_result' folder.")


27
image_paths 27
adfm.2018077880006
Processed images/adfm.201702654/adfm.2017026540008.jpg into test_result_v3_ckpt400/adfm.2017026540008
pred [('HeaderMatter', 84, 36, 991, 120, 1), ('text', 87, 135, 536, 1247, 2), ('Heading', 89, 1262, 257, 1280, 3), ('text', 86, 1294, 536, 1316, 4), ('text', 541, 139, 998, 342, 5), ('FooterMatter', 90, 1352, 997, 1378, 6)]


/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/806577179.py:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = gt_draw.textsize(label_text, font=FONT)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/806577179.py:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = gt_draw.textsize(label_text, font=FONT)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/806577179.py:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = gt_draw.textsize(label_text, font=FONT)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/806577179.py:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  

Annotated images saved for images/adfm.201702654/adfm.2017026540008.jpg
adfm.2017026540008
Processed images/adfm.201702654/adfm.2017026540009.jpg into test_result_v3_ckpt400/adfm.2017026540009
pred [('HeaderMatter', 84, 36, 990, 120, 1), ('Figure', 91, 135, 997, 801, 2), ('FigureCaption', 83, 817, 991, 861, 3), ('text', 83, 861, 529, 1158, 4), ('Heading', 83, 1198, 419, 1244, 5), ('text', 84, 1266, 534, 1316, 6), ('text', 553, 86, 997, 1316, 7), ('FooterMatter', 85, 1352, 994, 1378, 8)]


/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/806577179.py:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = gt_draw.textsize(label_text, font=FONT)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/806577179.py:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = gt_draw.textsize(label_text, font=FONT)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/806577179.py:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = gt_draw.textsize(label_text, font=FONT)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/806577179.py:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  

Annotated images saved for images/adfm.201702654/adfm.2017026540009.jpg
adfm.2017026540009
Processed images/adfm.201702654/adfm.2017026540003.jpg into test_result_v3_ckpt400/adfm.2017026540003
pred [('HeaderMatter', 84, 36, 991, 120, 1), ('Figure', 213, 124, 865, 413, 2), ('FigureCaption', 79, 426, 993, 479, 3), ('text', 79, 496, 533, 612, 4), ('Heading', 85, 658, 419, 674, 5), ('text', 82, 691, 533, 1323, 6), ('Figure', 84, 136, 515, 459, 7), ('FigureCaption', 84, 474, 515, 613, 8), ('text', 548, 917, 991, 1322, 9), ('FooterMatter', 37, 1358, 991, 1384, 10)]


...

/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/806577179.py:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = gt_draw.textsize(label_text, font=FONT)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/806577179.py:222: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = pred_draw.textsize(label_text, font=FONT)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/806577179.py:222: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = pred_draw.textsize(label_text, font=FONT)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/806577179.py:222: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead

Annotated images saved for images/adfm.201807788/adfm.2018077880003.jpg
adfm.2018077880003
Processed images/adfm.201807788/adfm.2018077880006.jpg into test_result_v3_ckpt400/adfm.2018077880006
pred [('HeaderMatter', 84, 25, 990, 99, 1), ('Figure', 204, 119, 865, 630, 2), ('FigureCaption', 84, 637, 989, 680, 3), ('text', 84, 613, 533, 1323, 4), ('text', 542, 611, 996, 1322, 5), ('FooterMatter', 84, 1361, 989, 1384, 6)]


/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/806577179.py:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = gt_draw.textsize(label_text, font=FONT)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/806577179.py:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = gt_draw.textsize(label_text, font=FONT)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/806577179.py:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = gt_draw.textsize(label_text, font=FONT)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/806577179.py:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  

Annotated images saved for images/adfm.201807788/adfm.2018077880006.jpg
Test results with annotated images have been organized into the 'test_result' folder.


/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/806577179.py:222: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = pred_draw.textsize(label_text, font=FONT)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/806577179.py:222: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = pred_draw.textsize(label_text, font=FONT)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/806577179.py:222: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = pred_draw.textsize(label_text, font=FONT)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/806577179.py:222: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength inste

### to eval on train

In [41]:
import os
import json
import jsonlines
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont
import random

# Define paths
TEST_DATA_DIR = Path("train_data")
IMAGES_DIR = TEST_DATA_DIR
DATASET_JSON_PATH = TEST_DATA_DIR / "dataset.json"
# PREDICTIONS_JSONL_PATH = Path("generated_predictions.jsonl")  # Adjust if different
# PREDICTIONS_JSONL_PATH = Path("generated_predictions_v2_ckpt800.jsonl")
PREDICTIONS_JSONL_PATH = Path("generated_predictions_v3_ckpt400_train.jsonl")
# PREDICTIONS_JSONL_PATH = Path("generated_predictions_few_shot.jsonl")  # Adjust if different
# TEST_RESULT_DIR = Path("test_result_v2_ckpt800")
TEST_RESULT_DIR = Path("test_result_v3_ckpt400_train")

# Create the test_result directory if it doesn't exist
TEST_RESULT_DIR.mkdir(parents=True, exist_ok=True)

# Load dataset.json
with open(DATASET_JSON_PATH, 'r', encoding='utf-8') as f:
    dataset = json.load(f)

# Create a mapping from image path to ground truth
ground_truth_map = {}
unique_labels = set()  # To collect all unique labels

for entry in dataset:
    images = entry.get("images", [])
    # Assuming one image per entry; adjust if multiple
    for image_path in images:
        # Normalize image path
        image_path_normalized = image_path.replace("\\", "/")
        # Assuming the last conversation entry contains the ground truth as a list of tuples
        ground_truth = eval(entry["conversations"][-1]["value"])
        ground_truth_map[image_path_normalized] = ground_truth
        # Collect unique labels from ground truth
        for obj in ground_truth:
            if len(obj) >= 1:
                unique_labels.add(obj[0])

# Load predictions from JSONL
predictions = []
with jsonlines.open(PREDICTIONS_JSONL_PATH, 'r') as reader:
    for obj in reader:
        # Assuming 'predict' contains a string representation of a list of tuples
        predict_value = obj.get("predict", "")
        if isinstance(predict_value, str):
            try:
                prediction_tuple_list = eval(predict_value)
                predictions.append(prediction_tuple_list)
                # Collect unique labels from predictions
                for pred_obj in prediction_tuple_list:
                    if len(pred_obj) >= 1:
                        unique_labels.add(pred_obj[0])
            except Exception as e:
                print(f"Error evaluating prediction: {predict_value}")
                predictions.append([])
        elif isinstance(predict_value, list):
            # If 'predict' is already a list of tuples
            predictions.append(predict_value)
            # Collect unique labels from predictions
            for pred_obj in predict_value:
                if len(pred_obj) >= 1:
                    unique_labels.add(pred_obj[0])
        else:
            predictions.append([])
print(len(predictions))
if len(predictions) != len(ground_truth_map):
    print("Warning: Number of predictions does not match number of ground truth entries.")

# Now, iterate over images and create the test_result structure
image_paths = list(ground_truth_map.keys())
prediction_objects = predictions[:len(image_paths)]  # Truncate if necessary

# Define a list of distinguishable colors
PREDEFINED_COLORS = [
    "red", "green", "blue", "orange", "purple", "cyan",
    "magenta", "yellow", "lime", "pink", "brown", "teal",
    "olive", "maroon", "navy", "grey", "black"
]

# Initialize label to color mapping
label_color_map = {}
color_index = 0

# Function to assign a color to a label
def get_label_color(label):
    global color_index
    if label in label_color_map:
        return label_color_map[label]
    else:
        if color_index < len(PREDEFINED_COLORS):
            color = PREDEFINED_COLORS[color_index]
            color_index += 1
        else:
            # Generate a random color if predefined colors are exhausted
            color = "#{:06x}".format(random.randint(0, 0xFFFFFF))
        label_color_map[label] = color
        return color

# Load a font for labels
try:
    FONT = ImageFont.truetype("arial.ttf", size=15)
except IOError:
    FONT = ImageFont.load_default()
print('image_paths', len(image_paths))
for idx, image_rel_path in enumerate(image_paths):
    
    print(image_name)
    image_name = Path(image_rel_path).stem  # e.g., adfm.2017026540008
    image_folder = TEST_RESULT_DIR / image_name
    image_folder.mkdir(parents=True, exist_ok=True)

    # Ground truth
    ground_truth = ground_truth_map[image_rel_path]  # List of tuples
    ground_truth_path = image_folder / "ground_truth.txt"  # Changed from .json to .txt
    try:
        with open(ground_truth_path, 'w', encoding='utf-8') as gt_file:
            gt_file.write(str(ground_truth))  # Save as string representation
    except IOError as e:
        print(f"Error writing ground truth for {image_rel_path}: {e}")

    # Prediction
    if idx < len(prediction_objects):
        prediction = prediction_objects[idx]  # Already a list of tuples
    else:
        prediction = []
        print(f"No prediction found for image: {image_rel_path}")
    
    predicted_path = image_folder / "predicted.txt"  # Changed from .json to .txt
    try:
        with open(predicted_path, 'w', encoding='utf-8') as pred_file:
            pred_file.write(str(prediction))  # Save as string representation
    except IOError as e:
        print(f"Error writing prediction for {image_rel_path}: {e}")

    print(f"Processed {image_rel_path} into {image_folder}")

    # Paths to the original image
    original_image_path = IMAGES_DIR / image_rel_path
    if not original_image_path.exists():
        print(f"Original image not found: {original_image_path}")
        continue
    print('pred', prediction)
    # Open the original image
    with Image.open(original_image_path).convert("RGB") as img:
        # Create copies for ground truth and prediction
        gt_img = img.copy()
        pred_img = img.copy()

        gt_draw = ImageDraw.Draw(gt_img)
        pred_draw = ImageDraw.Draw(pred_img)

        # Draw Ground Truth Bounding Boxes
        for obj in ground_truth:
            # Each obj is a tuple: (label, x1, y1, x2, y2, reading_order)
            if len(obj) != 6:
                print(f"Invalid ground truth tuple format in {image_rel_path}: {obj}")
                continue
            label, x1, y1, x2, y2, reading_order = obj
            try:
                x1, y1, x2, y2 = float(x1), float(y1), float(x2), float(y2)
            except ValueError:
                print(f"Invalid coordinates in ground truth tuple in {image_rel_path}: {obj}")
                continue

            # Get color for the label
            color = get_label_color(label)

            # Draw rectangle
            gt_draw.rectangle([x1, y1, x2, y2], outline=color, width=2)

            # Prepare label text with reading order
            label_text = f"{label} ({reading_order})"

            # Calculate text size to create a filled rectangle behind the text for better visibility
            text_size = gt_draw.textsize(label_text, font=FONT)
            text_background = [x1, y1 - 15 if y1 - 15 > 0 else y1 + 5, x1 + text_size[0], y1]
            gt_draw.rectangle(text_background, fill=color)

            # Draw label text
            text_position = (x1, y1 - 15 if y1 - 15 > 0 else y1 + 5)
            gt_draw.text(text_position, label_text, fill="white", font=FONT)

        # Save Ground Truth Annotated Image
        gt_image_path = image_folder / "ground_truth_annotated.png"
        try:
            gt_img.save(gt_image_path, format="PNG")
        except IOError as e:
            print(f"Error saving ground truth annotated image for {image_rel_path}: {e}")

        # Draw Predicted Bounding Boxes
        for obj in prediction:
            # Each obj is a tuple: (label, x1, y1, x2, y2, reading_order)
            if len(obj) != 6:
                print(f"Invalid prediction tuple format in {image_rel_path}: {obj}")
                continue
            label, x1, y1, x2, y2, reading_order = obj
            try:
                x1, y1, x2, y2 = float(x1), float(y1), float(x2), float(y2)
            except ValueError:
                print(f"Invalid coordinates in prediction tuple in {image_rel_path}: {obj}")
                continue

            if x1 > x2 or y1 > y2:
                print(f"Invalid bounding box with x1 > x2 or y1 > y2 in prediction for {image_rel_path}: {obj}")
                continue

            # Get color for the label
            color = get_label_color(label)

            # Draw rectangle
            pred_draw.rectangle([x1, y1, x2, y2], outline=color, width=2)

            # Prepare label text with reading order
            label_text = f"{label} ({reading_order})"

            # Calculate text size to create a filled rectangle behind the text for better visibility
            text_size = pred_draw.textsize(label_text, font=FONT)
            text_background = [x1, y1, x1 + text_size[0], y1  + text_size[1]]
            pred_draw.rectangle(text_background, fill=color)

            # Draw label text
            text_position = (x1, y1)
            pred_draw.text(text_position, label_text, fill="white", font=FONT)

        # Save Predicted Annotated Image
        pred_image_path = image_folder / "predicted_annotated.png"
        try:
            pred_img.save(pred_image_path, format="PNG")
        except IOError as e:
            print(f"Error saving predicted annotated image for {image_rel_path}: {e}")

        print(f"Annotated images saved for {image_rel_path}")

print("Test results with annotated images have been organized into the 'test_result' folder.")


Error evaluating prediction: [('Figure', 64, 65, 1016, 591, 1), ('FigureCaption', 64, 591, 1016, 659, 2), ('text', 64, 65, 1016, 717, 3), ('MathBlock', 236, 741, 342, 935, 4), ('text', 64, 935, 1016, 1016, 5), ('MathBlock', 153, 1041, 415, 1064, 6), ('Heading', 64, 1066, 416, 1088, 7), ('text', 64, 1088, 537, 1162, 8), ('MathBlock', 153, 1162, 466, 1219, 9), ('text', 565, 131, 1016, 200, 10), ('MathBlock', 717, 205, 864, 226, 1191, 10), ('text', 565, 1191, 1016, 1219, 10), ('Heading', 565, 1241, 917, 1261, 118, 1269, 1296, 1221, 1348, 1315, 1384, 1355, 1355, 1384, 1386, 1355, 1352, 1384, 1375, 1355, 1355, 1384, 1386, 1355, 1394, 1384, 1494, 1494, 1355, 1394, 1386, 1494, 1355, 1394, 1494, 1386, 1384, 1496, 1394, 1354, 149, 1386, 1384, 1496, 1394, 1386, 1498, 1394, 1496, 1384, 1384, 1498, 1394, 1386, 1498, 1384, 1498, 1394, 1384, 1498, 1394, 1386, 1498, 1394, 1498, 1384, 1355, 149, 1386, 1394, 1498, 1384, 1386, 149, 1394, 1386, 149, 1394, 149, 1386, 1384, 149, 1394, 1386, 149, 1384, 149,

/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/1831341834.py:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = gt_draw.textsize(label_text, font=FONT)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/1831341834.py:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = gt_draw.textsize(label_text, font=FONT)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/1831341834.py:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = gt_draw.textsize(label_text, font=FONT)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/1831341834.py:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead

Annotated images saved for images/adfm.201702654/adfm.2017026540010.jpg
adfm.2017026540010
Processed images/adfm.201702654/adfm.2017026540004.jpg into test_result_v3_ckpt400_train/adfm.2017026540004
pred [('HeaderMatter', 88, 35, 995, 105, 1), ('text', 89, 133, 534, 1318, 2), ('text', 550, 139, 999, 336, 3), ('Heading', 552, 377, 678, 395, 4), ('text', 553, 401, 996, 422, 5), ('FooterMatter', 92, 1353, 995, 1372, 6)]


/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/1831341834.py:222: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = pred_draw.textsize(label_text, font=FONT)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/1831341834.py:222: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = pred_draw.textsize(label_text, font=FONT)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/1831341834.py:222: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = pred_draw.textsize(label_text, font=FONT)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/1831341834.py:222: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength i

Annotated images saved for images/adfm.201702654/adfm.2017026540004.jpg
adfm.2017026540004
Processed images/adfm.201702654/adfm.2017026540005.jpg into test_result_v3_ckpt400_train/adfm.2017026540005
pred [('HeaderMatter', 84, 36, 991, 120, 1), ('Figure', 213, 124, 865, 439, 2), ('FigureCaption', 79, 449, 993, 482, 3), ('text', 79, 512, 533, 1323, 4), ('text', 553, 510, 999, 648, 5), ('Heading', 553, 681, 675, 708, 6), ('text', 553, 714, 996, 1322, 7), ('FooterMatter', 90, 1361, 995, 1384, 8)]


...

Annotated images saved for images/adfm.201807034/adfm.2018070340002.jpg
adfm.2018070340002
Processed images/adfm.201807034/adfm.2018070340003.jpg into test_result_v3_ckpt400_train/adfm.2018070340003
pred [('HeaderMatter', 84, 25, 990, 95, 1), ('text', 77, 123, 530, 1147, 2), ('Heading', 81, 1178, 420, 1204, 3), ('text', 83, 1218, 534, 1326, 4), ('text', 543, 122, 996, 1327, 5), ('FooterMatter', 83, 1358, 989, 1384, 6)]


/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/1831341834.py:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = gt_draw.textsize(label_text, font=FONT)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/1831341834.py:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = gt_draw.textsize(label_text, font=FONT)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/1831341834.py:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = gt_draw.textsize(label_text, font=FONT)
/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/1831341834.py:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead

Annotated images saved for images/adfm.201807034/adfm.2018070340003.jpg
adfm.2018070340003
Processed images/adfm.201807034/adfm.2018070340000.jpg into test_result_v3_ckpt400_train/adfm.2018070340000
pred [('HeaderMatter', 84, 15, 990, 99, 1), ('Title', 86, 123, 953, 259, 2), ('text', 86, 274, 953, 369, 3), ('text', 78, 396, 678, 888, 4), ('Heading', 89, 921, 225, 947, 5), ('text', 89, 964, 993, 1083, 6), ('text', 88, 1098, 996, 1165, 7), ('text', 88, 1234, 998, 1326, 8), ('FooterMatter', 90, 1361, 995, 1384, 9)]


/var/folders/xs/v7l897x546q6cn033h14xvzc0000gn/T/ipykernel_19858/1831341834.py:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = gt_draw.textsize(label_text, font=FONT)


ValueError: y1 must be greater than or equal to y0

In [20]:
print(eval("[('HeaderMatter', 82, 26, 988, 99, 1), ('text', 85, 120, 531, 632, 2), ('text', 547, 122, 993, 630, 3), ('Heading', 548, 664, 732, 687, 4), ('text', 547, 700, 989, 738, 5), ('text', 549, 764, 989, 842, 6), ('FooterMatter', 82, 1358, 988, 1384, 7)]"))



[('HeaderMatter', 82, 26, 988, 99, 1), ('text', 85, 120, 531, 632, 2), ('text', 547, 122, 993, 630, 3), ('Heading', 548, 664, 732, 687, 4), ('text', 547, 700, 989, 738, 5), ('text', 549, 764, 989, 842, 6), ('FooterMatter', 82, 1358, 988, 1384, 7)]


In [18]:
print("""{\n    \"imageWidth\": 1080,\n    \"imageHeight\": 1440,\n    \"objects\": [\n        {\n            \"readingOrder\": 1,\n            \"label\": \"HeaderMatter\",\n            \"boundingBox\": {\n                \"x1\": 88,\n                \"y1\": 37,\n                \"x2\": 995,\n                \"y2\": 107\n            }\n        },\n        {\n            \"readingOrder\": 2,\n            \"label\": \"text\",\n            \"boundingBox\": {\n                \"x1\": 88,\n                \"y1\": 129,\n                \"x2\": 534,\n                \"y2\": 234\n            }\n        },\n        {\n            \"readingOrder\": 3,\n            \"label\": \"Heading\",\n            \"boundingBox\": {\n                \"x1\": 90,\n                \"y1\": 274,\n                \"x2\": 501,\n                \"y2\": 313\n            }\n        },\n        {\n            \"readingOrder\": 4,\n            \"label\": \"text\",\n            \"boundingBox\": {\n                \"x1\": 83,\n                \"y1\": 330,\n                \"x2\": 532,\n                \"y2\": 1313\n            }\n        },\n        {\n            \"readingOrder\": 5,\n            \"label\": \"TableCaption\",\n            \"boundingBox\": {\n                \"x1\": 555,\n                \"y1\": 137,\n                \"x2\": 996,\n                \"y2\": 195\n            }\n        },\n        {\n            \"readingOrder\": 6,\n            \"label\": \"Table\",\n            \"boundingBox\": {\n                \"x1\": 550,\n                \"y1\": 203,\n                \"x2\": 997,\n                \"y2\": 307\n            }\n        },\n        {\n            \"readingOrder\": 7,\n            \"label\": \"text\",\n            \"boundingBox\": {\n                \"x1\": 549,\n                \"y1\": 337,\n                \"x2\": 998,\n                \"y2\": 1311\n            }\n        },\n        {\n            \"readingOrder\": 8,\n            \"label\": \"FooterMatter\",\n            \"boundingBox\": {\n                \"x1\": 88,\n                \"y1\": 1351,\n                \"x2\": 992,\n                \"y2\": 1371\n            }\n        }\n    ]\n}""")

{
    "imageWidth": 1080,
    "imageHeight": 1440,
    "objects": [
        {
            "readingOrder": 1,
            "label": "HeaderMatter",
            "boundingBox": {
                "x1": 88,
                "y1": 37,
                "x2": 995,
                "y2": 107
            }
        },
        {
            "readingOrder": 2,
            "label": "text",
            "boundingBox": {
                "x1": 88,
                "y1": 129,
                "x2": 534,
                "y2": 234
            }
        },
        {
            "readingOrder": 3,
            "label": "Heading",
            "boundingBox": {
                "x1": 90,
                "y1": 274,
                "x2": 501,
                "y2": 313
            }
        },
        {
            "readingOrder": 4,
            "label": "text",
            "boundingBox": {
                "x1": 83,
                "y1": 330,
                "x2": 532,
                "y2": 1313
            }
        },
 